# <b><center>Feature Engineering</center></b>

In [2]:
! pip install tensorflow
# # ! pip install keras
# ! pip install -q keras
# ! pip install -q tensorflow
# ! pip install pandas

  Using cached tensorflow-2.13.0-cp311-cp311-macosx_12_0_arm64.whl (1.9 kB)
  Using cached tensorflow_macos-2.13.0-cp311-cp311-macosx_12_0_arm64.whl (189.3 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached libclang-16.0.6-py2.py3-none-macosx_11_0_arm64.whl (20.6 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached protobuf-4.23.4-cp37-abi3-macosx_10_9_universal2.whl (400 kB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Using cached grpcio-1.56.2-cp311-cp311-macosx_10_10_universal2.whl (8.9 MB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl (440 kB)
  Using

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
# Load the VGG16 model
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 [==============================] - 4s 0us/step


In [7]:
df__bk_rat = pd.read_csv(r'/Users/royzalta/Downloads/archive/Book reviews/Book reviews/BX-Book-Ratings.csv', sep=';', encoding="latin-1")
df_bk_dt = pd.read_csv(r'/Users/royzalta/Downloads/archive/Book reviews/Book reviews/BX_Books.csv', sep=';', encoding="latin-1")
df_usr_dt = pd.read_csv('/Users/royzalta/Downloads/archive/Book reviews/Book reviews/BX-Users.csv', sep=';', encoding="latin-1")
prepo_df = pd.read_csv(r'/Users/royzalta/Downloads/archive/Books Data with Category Language and Summary/Preprocessed_data.csv')

In [11]:
df_bk_dt.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [10]:
#extract features from book image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

def extract_features(image_path, model):
    img = preprocess_image(image_path)
    features = model.predict(img)
    return features.flatten()


In [ ]:
# Assuming you have a DataFrame called 'books_data' containing your book data
features_list = []
for image_url in df_bk_dt['Image-URL-S']:
    try:
        features = extract_features(image_url, vgg16_model)
        features_list.append(features)
    except Exception as e:
        print(f"Error processing image: {image_url}")


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Error processing image: http://images.amazon.com/images/P/3423032383.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/3442436060.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/3800026589.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/3800029871.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0143012282.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0140231021.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0312208928.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0374138230.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0374308276.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0380773007.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0385140037.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0394837

In [13]:
df_bk_dt.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [14]:
import requests
from PIL import Image
from io import BytesIO

def extract_features(image_url, model):
    try:
        # Download the image from the URL
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))

        # Preprocess the image and extract features
        img = img.resize((224, 224))  # Resize to match VGG16 input size
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        features = model.predict(img)
        return features.flatten()
    except Exception as e:
        print(f"Error processing image: {image_url}")
        return None


In [ ]:
# Assuming you have a DataFrame called 'books_data' containing your book data
features_list = []
for image_url in df_bk_dt['Image-URL-S']:
    try:
        features = extract_features(image_url, vgg16_model)
        features_list.append(features)
    except Exception as e:
        print(f"Error processing image: {image_url}")


Error processing image: http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0002005018.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0060973129.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0374157065.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0393045218.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0399135782.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0425176428.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0671870432.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0679425608.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/074322678X.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0771074670.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0806521

Error processing image: http://images.amazon.com/images/P/0385235941.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0446677450.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0451166892.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0553347594.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0671621009.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/067976397X.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0684822733.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0786868716.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/8472238822.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0671864769.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0671521519.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0440222

Error processing image: http://images.amazon.com/images/P/3548254268.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/3570300099.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/3596292646.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0060977493.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0312872682.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0316602906.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0316693006.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0345311396.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0345378490.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0345404793.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0345433173.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0375400

Error processing image: http://images.amazon.com/images/P/1563892278.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/1563898586.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/156971620X.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/1592571301.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0374270325.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0375719180.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0440235502.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/044023722X.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/1573225487.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/1853260673.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0060977477.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0345391

Error processing image: http://images.amazon.com/images/P/0198320264.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0373196989.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0373245580.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0451527747.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0671867113.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0812504208.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0060173289.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0060973897.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/006101351X.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0140104682.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0312143370.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0316666

...

Error processing image: http://images.amazon.com/images/P/0743428684.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0525947655.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0312316690.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0060005556.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0312304269.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0399148760.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0316603287.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0399145664.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0425178102.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0553580515.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0749301015.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0445204

Error processing image: http://images.amazon.com/images/P/0140193669.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0140194916.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0140255109.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/014070731X.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0140445145.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0452010845.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0140048162.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0140390537.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0192713434.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0345396936.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0312261918.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0140108

Error processing image: http://images.amazon.com/images/P/0553279556.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0812580141.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0374281947.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0345420438.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0689300549.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0345384911.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0345332997.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0451194101.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0451191013.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0553564943.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0886772370.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0671637

Error processing image: http://images.amazon.com/images/P/0590523465.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0140319301.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0816728798.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0440472091.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0439136369.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0688160867.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0590673181.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0380712369.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0679804021.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0671798340.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0020418809.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0449701

Error processing image: http://images.amazon.com/images/P/1853260045.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/2266051326.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0752860178.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/849515708X.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/8488730179.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/8488413750.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0441009239.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0441008534.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0312924801.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/1857970446.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0786702427.01.THUMBZZZ.jpg
Error processing image: http://images.amazon.com/images/P/0590477